# Equity Linked Notes - Autocallable Athena (single)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents an Athena structure with single underlying.

**You will be able to:**
* Define Athena instrument with basic parameters
* Configure pricing parameters
* Evaluate Athena Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Athena, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Athena",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Underlying", type = "string", value="ASML_AS"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="05/08/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="19/08/2030"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="05/08/2030"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["05/08/2026", "05/08/2026", "19/08/2026", "19/08/2026"],
                    ["05/08/2027", "05/08/2027", "19/08/2027", "19/08/2027"],
                    ["07/08/2028", "07/08/2028", "21/08/2028", "21/08/2028"],
                    ["06/08/2029", "06/08/2029", "20/08/2029", "20/08/2029"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="80%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[["05/08/2025","100%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="70%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="4.5%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="0")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Index",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(Underlying)",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "If(Performance[t] >= Interpol(AutocallBarrier,PS()), 0, Alive[LastDate])",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive $n*CouponRate * Early[t]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, $n*CouponRate, 0) * Alive[LastDate])",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

athena = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

athena_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 10, 21),  # Set your desired valuation date
    report_ccy="USD",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "ASML.AS",
                  underlying_name = "ASML_AS",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing
try:
    response = sp.price(
        definitions=[athena],
        pricing_preferences=athena_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Athena",
    "dealCcy": "EUR",
    "reportCcy": "USD",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 0.393693416,
        "Count": 13.978968792,
        "Coupon": 0.046483822,
        "Early": 0.902413506,
        "Index": 4292.791096264,
        "OptionAtMaturity": -0.04246765,
        "Performance": 7.189400597,
        "Price": 979.818785818,
        "PriceIn%": 97.981878582,
        "Settlement": 0.975802614
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_athena_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_athena_valuation)

,Field,Value
0,marketValueInDealCcy,979.818786
1,marketValueInReportCcy,1136.540801


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=athena_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,ASML_AS,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-08-05,1.0000,597.100000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2026-08-05,0.9847,896.686138,1.501735,0.1386,0.8614,0.038735,0.860786,0.000000,0.000000,EUR,Future
2,2027-08-05,0.9664,906.895979,1.518834,0.1018,0.0368,0.003310,0.036773,0.000000,0.000000,EUR,Future
3,2028-08-07,0.9466,907.558293,1.519944,0.0876,0.0142,0.001915,0.014188,0.000000,0.000000,EUR,Future
4,2029-08-06,0.9257,912.729774,1.528605,0.0820,0.0056,0.001007,0.005595,0.000000,0.000000,EUR,Future
5,2030-08-05,0.9037,917.678231,1.536892,0.0000,0.0000,0.002652,0.081922,-0.046991,1084.184235,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_athena_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])
display(df_athena_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,2.700340
1,deltaAmountInReportCcy,3.132259
2,gammaAmountInDealCcy,0.591517
3,gammaAmountInReportCcy,0.686130
4,vegaAmountInDealCcy,-4.251470
5,vegaAmountInReportCcy,-4.931493
6,thetaAmountInDealCcy,0.364024
7,thetaAmountInReportCcy,0.422250
